In [7]:
import numpy as np

In [2]:
a = FID(5, "x", qubits = [1,2])
print(a)

NameError: name 'FID' is not defined

In [3]:
def FID(time, initial_phi = 0, measurment_basis = "x", qubits = [0,1,2,3,4]):
    # initial state
    qasm = "{"
    for q in qubits:
        if q == qubits[-1]:
            qasm += f'''Ry q[{q}], {3.14/2.}'''
        else:
            qasm += f'''Ry q[{q}], {3.14/2.} |'''
    qasm += "} \n"

    if initial_phi > 0:
        for q in qubits:
            if q == qubits[-1]:
                qasm += f'''Rz q[{q}], {initial_phi}'''
            else:
                qasm += f'''Rz q[{q}], {initial_phi} |'''
        qasm += "} \n"

    # waiting
    for q in qubits:
        qasm += f'''
        Wait q[{q}], {int(time)}'''  
    qasm += "\n {"
    
    
    # measurment
    for q in qubits:
        if q == qubits[-1]:
            qasm += f'''Measure_{measurment_basis} q[{q}]'''
        else:
            qasm += f'''Measure_{measurment_basis} q[{q}] | '''
    qasm += "} \n"    
    return qasm

# translate numbetr to binary
def translate_to_binary(number):
    return format(number, '05b')
def get_computational_basis(result):
    list_out = []
    for n in range(len(result['histogram'])):
        list_out.append([])
        for k in range(2**5):
            label = translate_to_binary(k)
            try:
                prob = result['histogram'][n][str(k)] 
            except:
                prob = 0
            list_out[-1].append((label, prob))
    return list_out


def get_expecitations(result):
    listout = get_computational_basis(result)
    probabilities = np.zeros((len(listout),2,5))
    for l,outcomes in enumerate(listout):
        for outcome in outcomes:
          
            res = np.array([int(x) for x in outcome[0]])
            for i in range(5):
                probabilities[l,res[i],i] += outcome[1]

    expectation_values = np.zeros((len(listout),5))
    for l in range(len(listout)):
        for i in range(5):
            expectation_values[l,i] = probabilities[l,0,i] - probabilities[l,-1,i]


    return expectation_values




In [4]:


from getpass import getpass
from coreapi.auth import BasicAuthentication
from quantuminspire.api import QuantumInspireAPI
from quantuminspire.qiskit import QI



#Connect to QI API, If no project has been initialized yet, uncomment the 3rd line to create one
server_url = r'https://api.quantum-inspire.com'
authentication = BasicAuthentication("j.a.krzywda@liacs.leidenuniv.nl","Rewolucja#93")
QI.set_authentication(authentication, project_name='python-test')
qi = QuantumInspireAPI(server_url, authentication, 'Qit')


backend_type = qi.get_backend_type_by_name('Starmon-5')

def send_circuit(circuit, shots):
    return qi.execute_qasm(circuit, backend_type=backend_type, number_of_shots=shots)

In [5]:

def batch(times, Nmax = 8, shots = 1024, qubits = [0,1,2,3,4]):
    intro = '''version 1.0
    qubits 5
    '''

    expect_values = np.zeros((3,5,len(times)))
    result_log = []
    circ = ""
    inds = []
    for nb,b in enumerate(["x","y","z"]):
        for tn,time in enumerate(times):
            circ += "{"
            for q in qubits:
                if q == qubits[-1]:
                    circ += f'''prep_z q[{q}]'''
                else:
                    circ += f'''prep_z q[{q}] |'''
            circ += "}\n"  
        
            circ += FID(time, measurment_basis = b, qubits = qubits)
            inds.append(tn)
            if len(inds) == Nmax or tn == len(times)-1:
                print(intro+circ)
                result = send_circuit(intro+circ, shots = shots)
                result_log.append(result)
                expect_values[nb,:,np.array(inds)] = get_expecitations(result)
                circ = ""
                inds = []

    return expect_values, result_log
            

In [6]:
times = np.linspace(0,150,15)
expect_values, result_log = batch(times, shots = 1024*2, qubits = [0,4])

TypeError: FID() got an unexpected keyword argument 'basis'

In [98]:
get_expecitations(result_log[0])

array([[0.98046875, 0.98632812, 0.98632812, 0.95703125, 0.9609375 ],
       [0.98828125, 0.98828125, 0.94726562, 0.92382812, 0.95898438],
       [0.98242188, 0.95507812, 0.90625   , 0.91210938, 0.97851562],
       [0.97265625, 0.953125  , 0.8125    , 0.91796875, 0.94921875],
       [0.97265625, 0.94140625, 0.70507812, 0.91015625, 0.95703125],
       [0.96289062, 0.88867188, 0.546875  , 0.85351562, 0.921875  ],
       [0.95117188, 0.87890625, 0.42578125, 0.8828125 , 0.8671875 ],
       [0.92773438, 0.81445312, 0.25      , 0.87109375, 0.859375  ]])

In [31]:

#Create the qasm code for the Quantum Computer/Simulator
x_measure = '''version 1.0
qubits 5
'''
times = np.linspace(0, 32, 9)[1:]
for time in times:
    x_measure += FID(time, basis="x")
    x_measure += f'''
    {{prep_z q[0] | prep_z q[1] | prep_z q[2] | prep_z q[3] | prep_z q[4]}}'''


#Create the qasm code for the Quantum Computer/Simulator
y_measure = '''version 1.0
qubits 5
'''
times = np.linspace(0, 32, 9)[1:]
for time in times:
    y_measure += FID(time, basis = "y")
    y_measure += f'''
    {{prep_z q[0] | prep_z q[1] | prep_z q[2] | prep_z q[3] | prep_z q[4]}}'''




#Select the backend and make the run.
#The result contains lots of text and URL's, but we're interested in the result of the run.
# This  can be found in the 'histogram', ordered as: [OrderedDict({'1': 1.0})]
resultx = qi.execute_qasm(x_measure, backend_type=backend_type, number_of_shots=1024*8)
resulty = qi.execute_qasm(y_measure, backend_type=backend_type, number_of_shots=1024*8)

In [27]:
# translate numbetr to binary
def translate_to_binary(number):
    return format(number, '05b')
def get_computational_basis(result):
    list_out = []
    for n in range(len(result['histogram'])):
        list_out.append([])
        for k in range(2**5):
            label = translate_to_binary(k)
            try:
                prob = result['histogram'][n][str(k)] 
            except:
                prob = 0
            list_out[-1].append((label, prob))
    return list_out


def get_expecitations(result):
    listout = get_computational_basis(result)
    probabilities = np.zeros((len(listout),2,5))
    for l,outcomes in enumerate(listout):
        for outcome in outcomes:
          
            res = np.array([int(x) for x in outcome[0]])
            for i in range(5):
                probabilities[l,res[i],i] += outcome[1]

    expectation_values = np.zeros((len(listout),5))
    for l in range(len(listout)):
        for i in range(5):
            expectation_values[l,i] = probabilities[l,0,i] - probabilities[l,-1,i]


    return expectation_values

In [9]:
expectx = get_expecitations(resultx)
expecty = get_expecitations(resulty)

NameError: name 'resultx' is not defined